In [ ]:
import requests
from IPython.display import display, HTML
from IPython.display import Image
from PIL import Image as PILImage
from io import BytesIO, StringIO
import bs4
import pandas as pd
import pytesseract


In [ ]:
session = requests.Session()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0',
}


## Login Page

In [ ]:
SRM_STUDENT_PORTAL_URI = 'https://sp.srmist.edu.in/srmiststudentportal/students/loginManager/youLogin.jsp'

response = session.get(
    'https://sp.srmist.edu.in/srmiststudentportal/students/loginManager/youLogin.jsp',
    headers=headers,
)

# session.cookies


In [ ]:
SRM_STUDENT_PORTAL_GET_CAPTCHA_URI = 'https://sp.srmist.edu.in/srmiststudentportal/captchas'

response = session.get(
    SRM_STUDENT_PORTAL_GET_CAPTCHA_URI,
    headers=headers,
)

capcha_bin = response.content
capcha_text = pytesseract.image_to_string(PILImage.open(BytesIO(capcha_bin))).strip()

# print(f'Capcha: {capcha_text}')
# display(Image(data=capcha_bin))


In [ ]:
data = {
    'txtPageAction': '1',
    'txtSK': '**',
    'txtAN': '**',
    'hdnCaptcha': capcha_text,
}

session.post(
    SRM_STUDENT_PORTAL_URI,
    data=data,
).text.find('Login Error : Invalid net id or password')


## Attendance

In [ ]:
ATTENDENCE_PAGE_URI = 'https://sp.srmist.edu.in/srmiststudentportal/students/report/studentAttendanceDetails.jsp'

response = session.post(
    ATTENDENCE_PAGE_URI,
)


In [ ]:
soup = bs4.BeautifulSoup(response.text, 'html.parser')
table = soup.findAll('table', class_='table')[0]

df = pd.read_html(StringIO(str(table)))[0]

# Remove rows whoose `Code` matches below list
code_blacklist = ['CL', 'Total']
df = df[~df['Code'].isin(code_blacklist)]
df


In [ ]:
soup = bs4.BeautifulSoup(response.text, 'html.parser')
table = soup.findAll('table', class_='table')[1]

df = pd.read_html(StringIO(str(table)))[0]
# month_map = {'JAN': 1, 'FEB': 2,'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12}
# Extracting the month and year where there are entries
months_years = []
for index, row in df.iterrows():
    month, year = row['Month / Year'].split(' / ')
    if row[['Present', 'Absent']].sum() > 0:
        months_years.append([int(year), pd.to_datetime(month, format='%b').month])

print(months_years)


In [ ]:
df = pd.read_html(StringIO(str(table)))[0]
for index, row in df.iterrows():
    mon, year = row['Month / Year'].split(' / ')
    print(mon, year)


In [ ]:
MONTHLY_ATTENDENCE_PAGE_URI = 'https://sp.srmist.edu.in/srmiststudentportal/students/report/studentAttendanceDetailsInner.jsp'
response = session.post(
    MONTHLY_ATTENDENCE_PAGE_URI,
    data={'ids': 1, 'attendanceMonth': 1, 'attendanceYear': 2024},
)
lst = []
import ast
df = pd.read_html(StringIO(str(response.text)))[0]
lst.extend(ast.literal_eval(df.to_json(index=False, orient='records')))
lst


## Timetable

In [ ]:
response = session.post(
    'https://sp.srmist.edu.in/srmiststudentportal/students/report/studentTimeTableDetails.jsp',
)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
timetable, legends = soup.findAll('table', class_='table')
timetable = pd.read_html(StringIO(str(timetable)))[0]
legends = pd.read_html(StringIO(str(legends)))[0]

# Clean the timetable dataframe
timetable.replace('-', pd.NA, inplace=True)
timetable.columns = timetable.columns.droplevel(0)
timetable.rename(columns={'Day order': 'Day'}, inplace=True)

# Clean the legends dataframe
legends.rename(columns={'Code': 'SubCode', 'Description': 'SubName'}, inplace=True)

# Create a map of subject name to subject code
sub_map = {data['SubCode'].strip(): data['SubName'].strip() for data in legends.to_dict(orient='records')}

# Replace the subject name with subject code in the timetable dataframe
for idx, row in timetable.iterrows():
    for col in timetable.columns:
        # If there is a single subject code in the cell
        if row[col] in sub_map:
            timetable.at[idx, col] = sub_map[row[col]]
        
        # For electives where there are multiple subject codes
        elif isinstance(row[col], str) and len(row[col].split(' ')) > 1:
            sub_codes = row[col].split(' ')
            sub_names = [sub_map[sub_code] for sub_code in sub_codes]
            timetable.at[idx, col] = ' / '.join(sub_names)

class_time = ['Day', '9:20-10:10', '10:10-11:00', '11:10-12:00', '12:00-12:50', '2:00-2:50', '2:50-3:40', '3:40-4:30']

timetable.drop('04:00-04:50', axis=1, inplace=True)
timetable.columns = class_time
timetable.drop(5, inplace=True)

new_timetable = {}
for item in timetable.to_dict(orient='records'):
    day = ''
    for key, value in item.items():
        if key == 'Day':
            day = value
            new_timetable[day] = {}
        else:
            new_timetable[day][key] = value
new_timetable


## Benchmarks

In [ ]:
# import requests
# url = "http://127.0.0.1:8000/get_attendance_details"
# payload = {'username': 'dm0359', 'password': 'Mymail@06'}


In [ ]:
# %%timeit -n 1 -r 20
# response = requests.request("POST", url, data=payload)
